In [4]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install kyber-py

Note: you may need to restart the kernel to use updated packages.


In [1]:
# -----------------------------------------------------------------------------
# CRYSTALS-KYBER POST-QUANTUM CRYPTOGRAPHY DEMO
# -----------------------------------------------------------------------------
#
# NOTE: This script requires a third-party library that implements Kyber.
# For this demonstration, we assume a compatible library is installed.
# You can install a common pure-Python implementation using pip:
#
# pip install kyber-py
#
# Kyber is a Key Encapsulation Mechanism (KEM) used to securely establish
# a shared secret key between two parties, typically Alice and Bob.
# We will use the Kyber768 parameters, which are recommended by NIST for
# security roughly equivalent to AES-192.
# -----------------------------------------------------------------------------

import os
# Flag to track if the required Kyber class was successfully imported
KYBER_CLASS = None

# We attempt to import the Kyber implementation.
# The error reported (NameError) suggests the import may fail, but the 
# script is still attempting to call the undefined class name.
try:
    from kyber_py.kyber768 import Kyber768
    KYBER_CLASS = Kyber768 # Assign the imported class reference to a dedicated variable
except ImportError:
    print("Error: The 'kyber-py' library is not installed.")
    print("Please install it using: pip install kyber-py")
    # We remove the hard 'exit(1)' here to allow a graceful fail message 
    # inside the main function, preventing the NameError.


def crystals_kyber_demo():
    """
    Demonstrates the three main steps of the Kyber KEM:
    1. Key Generation (Alice creates her public and private keys).
    2. Encapsulation (Bob uses the public key to generate a shared secret).
    3. Decapsulation (Alice uses her private key to recover the same shared secret).
    """
    
    # Check if the required Kyber class was successfully imported before proceeding.
    # This check prevents the 'NameError' if the import failed.
    if KYBER_CLASS is None:
        print("\nFATAL ERROR: Cannot run demo because the Kyber implementation is unavailable.")
        return

    print("--- 1. KYBER KEY GENERATION (ALICE) ---")
    # Initialize the Kyber KEM instance using the imported class reference
    kyber_kem = KYBER_CLASS()

    # Alice generates her key pair:
    # pk (Public Key): Sent to Bob for encryption.
    # sk (Secret Key): Kept private by Alice for decryption.
    # The key generation is deterministic if a seed is provided, but typically
    # uses a strong random number generator.
    try:
        pk, sk = kyber_kem.keygen()
        print(f"Key Pair Generated (Kyber768):")
        print(f"  Public Key (pk) size: {len(pk)} bytes")
        print(f"  Secret Key (sk) size: {len(sk)} bytes")
        print("Alice securely sends pk to Bob.")
        print("-" * 50)
    except Exception as e:
        print(f"Key Generation failed: {e}")
        return


    print("--- 2. KEY ENCAPSULATION (BOB) ---")
    # Bob receives Alice's Public Key (pk).
    # Bob performs the encapsulation, which generates two things:
    # 1. ct (Ciphertext): The encrypted 'package' sent back to Alice.
    # 2. ss_b (Shared Secret): The symmetric key Bob will use.
    try:
        # The 'encaps' function generates the secret key (ss) and the ciphertext (ct)
        ss_b, ct = kyber_kem.encaps(pk)
        
        print(f"Encapsulation Performed:")
        print(f"  Ciphertext (ct) size: {len(ct)} bytes")
        print(f"  Bob's Shared Secret (ss_b) generated. Size: {len(ss_b)} bytes")
        print("Bob securely sends ct to Alice.")
        print("-" * 50)
    except Exception as e:
        print(f"Encapsulation failed: {e}")
        return


    print("--- 3. KEY DECAPSULATION (ALICE) ---")
    # Alice receives the Ciphertext (ct) from Bob.
    # Alice uses her Secret Key (sk) and the Ciphertext (ct) to recover the 
    # same shared secret key (ss_a).
    try:
        ss_a = kyber_kem.decaps(sk, ct)
        
        print(f"Decapsulation Performed:")
        print(f"  Alice's Shared Secret (ss_a) recovered. Size: {len(ss_a)} bytes")
        print("-" * 50)
    except Exception as e:
        print(f"Decapsulation failed: {e}")
        return


    print("--- 4. VERIFICATION ---")
    # The crucial step: Do Alice and Bob have the same key?
    if ss_a == ss_b:
        print("✅ SUCCESS: The shared secrets match!")
        # Print the first few bytes of the key for visual confirmation
        print(f"   Shared Secret (Hex prefix): {ss_a[:16].hex()}...")
    else:
        # This should theoretically never happen unless an error occurred or
        # a key/ciphertext was tampered with.
        print("❌ FAILURE: The shared secrets DO NOT MATCH.")


if __name__ == "__main__":
    crystals_kyber_demo()

Error: The 'kyber-py' library is not installed.
Please install it using: pip install kyber-py

FATAL ERROR: Cannot run demo because the Kyber implementation is unavailable.
